In [1]:
import ee
import pandas as pd
import altair as alt
import numpy as np
import folium

In [2]:
ee.Authenticate()

True

In [3]:
ee.Initialize()

In [4]:
def create_reduce_region_function(geometry,
                                  reducer=ee.Reducer.mean(),
                                  scale=1000,
                                  crs='EPSG:4326',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4):
    def reduce_region_function(img):
        stat = img.reduceRegion(
        reducer=reducer,
        geometry=geometry,
        scale=scale,
        crs=crs,
        bestEffort=bestEffort,
        maxPixels=maxPixels,
        tileScale=tileScale)
        
        return ee.Feature(geometry, stat).set({'millis': img.date().millis()})
    
    return reduce_region_function


This function accepts an ee.Image object and reduces it over a specified region, according to the provided arguments. The reduced data is then packaged into an ee.Feature object, which also includes the timestamp of the image.(milliseconds from Unix epoch (included to enable time series plotting).)



In [5]:
# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

In [6]:
def fc_to_dict(fc):
  # Check if the FeatureCollection is empty
  if fc.size().getInfo() == 0:
    print("FeatureCollection is empty")
    return None

  # Get the property names of the first feature in the FeatureCollection
  prop_names = fc.first().propertyNames()

  # Check if the first feature has properties
  if prop_names.size().getInfo() == 0:
    print("First feature has no properties")
    return None

  # Reduce the FeatureCollection to lists of property values
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

  # Create a dictionary from the property names and lists of property values
  return ee.Dictionary.fromLists(prop_names, prop_lists)


 Function fc_to_dict  converts the properties of an ee.FeatureCollection into a dictionary.
 The function to converts the feature collection to an ee.Dictionary where the keys are feature property names and values are corresponding lists of property values, which pandas can deal with handily.

Extract the property values from the ee.FeatureCollection as a list of lists stored in an ee.Dictionary using reduceColumns().
Extract the list of lists from the dictionary.
Add names to each list by converting to an ee.Dictionary where keys are property names and values are the corresponding value lists.
 

## time series

In [10]:
#load data and filter to aoi
today = ee.Date(pd.to_datetime('today'))#creates an ee.Date object for the current date
date_range = ee.DateRange(today.advance(-20, 'years'), today)# ee.DateRange object that represents the last 20 years.
spei = ee.ImageCollection("CSIC/SPEI/2_9").filterDate(date_range).select('SPEI_01_month')#
aoi = ee.FeatureCollection("projects/connect-gaea/assets/project_areas").geometry()

## Reduce data
Create a region reduction function.
Map the function over the pdsi image collection to reduce each image.
Filter out any resulting features that have null computed values (occurs when all pixels in an AOI are masked).

In [11]:
#get infor on the image 
spei_within_aoi = spei.filterBounds(aoi)

# Get the first image in the filtered collection
first_image_within_aoi = spei_within_aoi.first()

# Get the band names of the first image
band_names_within_aoi = first_image_within_aoi.bandNames()

# Print the band names
print("Band Names within AOI:", band_names_within_aoi.getInfo())

Band Names within AOI: ['SPEI_01_month']


In [12]:
reduce_spei = create_reduce_region_function(
    geometry=aoi, reducer=ee.Reducer.mean(), scale=5000, crs='EPSG:3310')

spei_stat_fc = ee.FeatureCollection(spei.map(reduce_spei)).filter(
    ee.Filter.notNull(spei.first().bandNames()))

In [13]:
if spei_stat_fc is None:
    print("spei_stat_fc is None")
else:
    print("spei_stat_fc is not None")


spei_stat_fc is not None


In [14]:
spei_dict = fc_to_dict(spei_stat_fc)
if spei_dict is not None:
    spei_dict = spei_dict.getInfo()
else:
    print("fc_to_dict(spei_stat_fc) returned None")


In [16]:
print(type(spei_dict), '\n')
for prop in spei_dict.keys():
    print(prop + ':', spei_dict[prop][0:3] + ['...'])

<class 'dict'> 

SPEI_01_month: [-0.1804162897590727, -1.3905571184450782, -0.24125110097723848, '...']
millis: [1086048000000, 1088640000000, 1091318400000, '...']
system:index: ['2004_06_01', '2004_07_01', '2004_08_01', '...']


In [18]:
#convert to df
spei_df = pd.DataFrame(spei_dict)

#display
display(spei_df)
print(spei_df.dtypes)

,SPEI_01_month,millis,system:index
0,-0.180416,1086048000000,2004_06_01
1,-1.390557,1088640000000,2004_07_01
2,-0.241251,1091318400000,2004_08_01
3,0.277465,1093996800000,2004_09_01
4,0.082952,1096588800000,2004_10_01
...,...,...,...
218,2.643881,1659312000000,2022_08_01
219,2.119641,1661990400000,2022_09_01
220,-0.348979,1664582400000,2022_10_01
221,-0.606635,1667260800000,2022_11_01


SPEI_01_month    float64
millis             int64
system:index      object
dtype: object


In [19]:
#add column date

# Function to add date variables to DataFrame.
def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
  df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  return df

In [21]:
pdsi_df = add_date_info(spei_df)
pdsi_df.head(10)

,SPEI_01_month,millis,system:index,Timestamp,Year,Month,Day,DOY
0,-0.180416,1086048000000,2004_06_01,2004-06-01,2004,6,1,153
1,-1.390557,1088640000000,2004_07_01,2004-07-01,2004,7,1,183
2,-0.241251,1091318400000,2004_08_01,2004-08-01,2004,8,1,214
3,0.277465,1093996800000,2004_09_01,2004-09-01,2004,9,1,245
4,0.082952,1096588800000,2004_10_01,2004-10-01,2004,10,1,275
5,0.456167,1099267200000,2004_11_01,2004-11-01,2004,11,1,306
6,0.071609,1101859200000,2004_12_01,2004-12-01,2004,12,1,336
7,0.074103,1104537600000,2005_01_01,2005-01-01,2005,1,1,1
8,0.019326,1107216000000,2005_02_01,2005-02-01,2005,2,1,32
9,-0.161718,1109635200000,2005_03_01,2005-03-01,2005,3,1,60


In [22]:
#some house cleaning
spei_df = spei_df.rename(columns={
    'SPEI_01_month': 'SPEI'
}).drop(columns=['millis', 'system:index'])
spei_df.head(5)

,SPEI,Timestamp,Year,Month,Day,DOY
0,-0.180416,2004-06-01,2004,6,1,153
1,-1.390557,2004-07-01,2004,7,1,183
2,-0.241251,2004-08-01,2004,8,1,214
3,0.277465,2004-09-01,2004,9,1,245
4,0.082952,2004-10-01,2004,10,1,275


In [24]:
#check data type
spei_df.dtypes

SPEI                float64
Timestamp    datetime64[ns]
Year                  int32
Month                 int32
Day                   int32
DOY                   int32
dtype: object

## visualisation

In [ ]:
#+VE = wet conditions -Ve= dry conditions

In [28]:
alt.Chart(spei_df).mark_rect().encode(
    x='Year:O',
    y='Month:O',
    color=alt.Color(
        'mean(SPEI):Q', scale=alt.Scale(scheme='redblue', domain=(-2.5, 2.5))),
    tooltip=[
        alt.Tooltip('Year:O', title='Year'),
        alt.Tooltip('Month:O', title='Month'),
        alt.Tooltip('mean(SPEI):Q', title='SPEI')
    ]).properties(width=600, height=300)

alt.Chart(...)

In [29]:
#bar chart

alt.Chart(spei_df).mark_bar(size=1).encode(
    x='Timestamp:T',
    y='SPEI:Q',
    color=alt.Color(
        'SPEI:Q', scale=alt.Scale(scheme='redblue', domain=(-2.5, 2.5))),
    tooltip=[
        alt.Tooltip('Timestamp:T', title='Date'),
        alt.Tooltip('SPEI:Q', title='SPEI')
    ]).properties(width=600, height=300)

alt.Chart(...)

## VEGETATION INDICES

MODIS provides an analysis-ready 16-day NDVI composite that is well suited for regional investigation of temporal vegetation dynamics

In [30]:
ndvi = ee.ImageCollection('MODIS/006/MOD13A2').filterDate(date_range).select('NDVI')

reduce_ndvi = create_reduce_region_function(
    geometry=aoi, reducer=ee.Reducer.mean(), scale=1000, crs='EPSG:3310')

ndvi_stat_fc = ee.FeatureCollection(ndvi.map(reduce_ndvi)).filter(
    ee.Filter.notNull(ndvi.first().bandNames()))

In [31]:
#data frame

ndvi_dict = fc_to_dict(ndvi_stat_fc).getInfo()
ndvi_df = pd.DataFrame(ndvi_dict)
display(ndvi_df)
print(ndvi_df.dtypes)

,NDVI,millis,system:index
0,6505.371882,1085356800000,2004_05_24
1,6501.088751,1086739200000,2004_06_09
2,6365.281425,1088121600000,2004_06_25
3,5841.363719,1089504000000,2004_07_11
4,5858.803742,1090886400000,2004_07_27
...,...,...,...
426,6027.906942,1670025600000,2022_12_03
427,5993.100137,1671408000000,2022_12_19
428,6122.643860,1672531200000,2023_01_01
429,5350.292243,1673913600000,2023_01_17


NDVI            float64
millis            int64
system:index     object
dtype: object


In [32]:
ndvi_df['NDVI'] = ndvi_df['NDVI'] / 10000
ndvi_df = add_date_info(ndvi_df)
ndvi_df.head(5)

,NDVI,millis,system:index,Timestamp,Year,Month,Day,DOY
0,0.650537,1085356800000,2004_05_24,2004-05-24,2004,5,24,145
1,0.650109,1086739200000,2004_06_09,2004-06-09,2004,6,9,161
2,0.636528,1088121600000,2004_06_25,2004-06-25,2004,6,25,177
3,0.584136,1089504000000,2004_07_11,2004-07-11,2004,7,11,193
4,0.585880,1090886400000,2004_07_27,2004-07-27,2004,7,27,209


In [57]:
highlight = alt.selection(
    type='single', on='mouseover', fields=['Year'], nearest=True)

base = alt.Chart(ndvi_df).encode(
    x=alt.X('DOY:Q', scale=alt.Scale(domain=[0, 353], clamp=True)),
    y=alt.Y('NDVI:Q', scale=alt.Scale(domain=[0.2, 0.8])),
    color=alt.Color('Year:O', scale=alt.Scale(scheme='magma')))

points = base.mark_circle().encode(
    opacity=alt.value(0),
    tooltip=[
        alt.Tooltip('Year:O', title='Year'),
        alt.Tooltip('DOY:Q', title='DOY'),
        alt.Tooltip('NDVI:Q', title='NDVI')
    ]).add_selection(highlight)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1), alt.value(3)))

(points + lines).properties(width=600, height=350).interactive()

C:\Users\Lucy Kimani\.conda\envs\gee\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection' is deprecated.
   Use 'selection_point()' or 'selection_interval()' instead; these functions also include more helpful docstrings.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
C:\Users\Lucy Kimani\.conda\envs\gee\Lib\site-packages\altair\vegalite\v5\api.py:469: AltairDeprecationWarning: The types 'single' and 'multi' are now
        combined and should be specified using "selection_point()".
  warnings.warn(
C:\Users\Lucy Kimani\.conda\envs\gee\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.LayerChart(...)

# Drought and Productivity

### Relationship btn drought and NDVI


In [ ]:
#Filter the NDVI DataFrame to observations that occur between DOY 224 and 272.
#Reduce the DOY-filtered subset to intra-annual minimum NDVI.

###  SPEI in the first part of the year influences NDVI in the later part of the 

How the SPEI from the start of the growing season (April) through the flowering period (August) influences the NDVI during the flowering period.

In [123]:
ndvi_doy_range = [120, 240]

ndvi_df_sub = ndvi_df[(ndvi_df['DOY'] >= ndvi_doy_range[0])
                      & (ndvi_df['DOY'] <= ndvi_doy_range[1])]

ndvi_df_sub = ndvi_df_sub.groupby('Year').min(numeric_only=True)

In [124]:
#Filter the PDSI DataFrame to observations that occur between DOY 1 and 272.
#Reduce the values within a given year to the mean of the observations

spei_doy_range = [60, 240]

spei_df_sub = spei_df[(spei_df['DOY'] >= spei_doy_range[0])
                      & (spei_df['DOY'] <= spei_doy_range[1])]

spei_df_sub = spei_df_sub.groupby('Year').mean(numeric_only=True)

In [125]:
#combine the two df

In [126]:
ndvi_spei_df = pd.merge(
    ndvi_df_sub, spei_df_sub, how='left', on='Year').reset_index()

ndvi_spei_df = ndvi_spei_df[['Year', 'NDVI', 'SPEI']]

ndvi_spei_df.head(10)

,Year,NDVI,SPEI
0,2004,0.571773,-0.604075
1,2005,0.611741,-0.429507
2,2006,0.600719,-0.289066
3,2007,0.643580,0.228677
4,2008,0.600166,-0.495238
5,2009,0.536721,-0.938047
6,2010,0.582415,0.176289
7,2011,0.580086,-0.185662
8,2012,0.639717,0.321003
9,2013,0.579389,0.174003


In [127]:
ndvi_spei_df['Fit'] = np.poly1d(
    np.polyfit(ndvi_spei_df['SPEI'], ndvi_spei_df['NDVI'], 1))(
        ndvi_spei_df['SPEI'])


In [128]:
ndvi_spei_df.head(5)

,Year,NDVI,SPEI,Fit
0,2004,0.571773,-0.604075,0.587935
1,2005,0.611741,-0.429507,0.586105
2,2006,0.600719,-0.289066,0.584632
3,2007,0.643580,0.228677,0.579203
4,2008,0.600166,-0.495238,0.586794


In [131]:
base = alt.Chart(ndvi_spei_df).encode(
    x=alt.X('SPEI:Q', scale=alt.Scale(domain=(-1.0, 1))))

points = base.mark_circle(size=60).encode(
    y=alt.Y('NDVI:Q', scale=alt.Scale(domain=(0.2, 0.8))),
    color=alt.Color('Year:O', scale=alt.Scale(scheme='magma')),
    tooltip=[
        alt.Tooltip('Year:O', title='Year'),
        alt.Tooltip('SPEI:Q', title='SPEI'),
        alt.Tooltip('NDVI:Q', title='NDVI')
    ])

fit = base.mark_line().encode(
    y=alt.Y('Fit:Q'),
    color=alt.value('#808080'))

(points + fit).properties(width=600, height=300).interactive()

alt.LayerChart(...)

### Positive Slope: 
If the fitted line slopes upwards, it suggests that as PDSI increases, NDVI also increases during the specified periods. This could indicate that higher PDSI values (i.e., wetter conditions) are associated with higher NDVI values (i.e., more vegetation or healthier vegetation) during those times of the year.
### Negative Slope: 
If the fitted line slopes downwards, it suggests that as PDSI increases, NDVI decreases during the specified periods. This could indicate that higher PDSI values (i.e., wetter conditions) are associated with lower NDVI values (i.e., less vegetation or less healthy vegetation) during those times of the year.
### Steepness of the Slope: 
The steepness of the slope indicates the strength of the relationship. A steeper slope indicates a stronger relationship between PDSI and NDVI.

In [138]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine, USDA National Agriculture Imagery Program</a>',
      name=name,
      overlay=True,
      control=True).add_to(self)

In [139]:
# Add an Earth Engine layer drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [145]:
# Import a NAIP image for the area and date of interest.
naip_img = ee.ImageCollection("ESA/WorldCereal/2021/MARKERS/v100").filterDate(
    '2020-01-01',
    '2021-12-31').filterBounds(ee.Geometry.Point([34.4977, 0.3818 ])).first()

In [146]:
# Display the NAIP image to the folium map.
m = folium.Map(location=[0.3818, 34.4977], tiles='OpenStreetMap', zoom_start=16)
m.add_ee_layer(naip_img, None, 'cereals image, 2016')

In [147]:
# Add the point of interest to the map.
folium.Circle(
    radius=15,
    location=[0.3818, 34.4977],
    color='yellow',
    fill=False,
).add_to(m)

# Add the AOI to the map.
folium.GeoJson(
  aoi.getInfo(),
  name='geojson',
  style_function=lambda x: {'fillColor': '#00000000', 'color': '#000000'},
).add_to(m)

# Add a lat lon popup.
folium.LatLngPopup().add_to(m)

# Display the map.
display(m)

### Define Landsat observation date window inputs based on NDVI curve plotted previously and set latitude and longitude variables from the map above.

In [148]:
start_day = 120
end_day = 240

latitude = 0.3818
longitude =  34.4977

In [152]:
# Make lat. and long. vars an `ee.Geometry.Point`.
point = ee.Geometry.Point([longitude, latitude])

# Define a function to get and rename bands of interest from OLI.
def rename_oli(img):
  return (img.select(
      ee.List(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']),
      ee.List(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'])))

# Define a function to get and rename bands of interest from ETM+.
def rename_etm(img):
  return (img.select(
      ee.List(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']),
      ee.List(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'])))

# Define a function to scale images and mask out clouds and cloud shadows.
def scale_and_mask(img):
    qa_mask = img.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    scaled = img.select('SR_B.').multiply(0.0000275).add(-0.2)
    return scaled.updateMask(qa_mask)

# Define a function to add year as an image property.
def set_year(img):
  year = ee.Image(img).date().get('year')
  return img.set('Year', year)

# Define a function to calculate NBR.
def calc_nbr(img):
  return img.normalizedDifference(ee.List(['NIR', 'SWIR2'])).rename('NBR')

# Define a function to prepare OLI images.
def prep_oli(img):
  orig = img
  img = scale_and_mask(img)
  img = rename_oli(img)
  img = calc_nbr(img)
  img = img.copyProperties(orig, orig.propertyNames())
  return set_year(img)

# Define a function to prepare TM/ETM+ images.
def prep_etm(img):
  orig = img
  img = scale_and_mask(img)
  img = rename_etm(img)
  img = calc_nbr(img)
  img = img.copyProperties(orig, orig.propertyNames())
  return set_year(img)

# Import image collections for each Landsat sensor (surface reflectance).
tm_col = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
etm_col = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')
oli_col = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')

# Filter collections and prepare them for merging.
oli_col = oli_col.filterBounds(point).filter(
    ee.Filter.calendarRange(start_day, end_day, 'day_of_year')).map(prep_oli)

etm_col = etm_col.filterBounds(point).filter(
    ee.Filter.calendarRange(start_day, end_day, 'day_of_year')).map(prep_etm)

tm_col = tm_col.filterBounds(point).filter(
    ee.Filter.calendarRange(start_day, end_day, 'day_of_year')).map(prep_etm)

# Merge the collections.
landsat_col = oli_col.merge(etm_col).merge(tm_col)

# Get a distinct year collection.
distinct_year_col = landsat_col.distinct('Year')

# Define a filter that identifies which images from the complete collection
# match the year from the distinct year collection.
join_filter = ee.Filter.equals(leftField='Year', rightField='Year')

# Define a join.
join = ee.Join.saveAll('year_matches')

# Apply the join and convert the resulting FeatureCollection to an
# ImageCollection.
join_col = ee.ImageCollection(
    join.apply(distinct_year_col, landsat_col, join_filter))

# Define a function to apply mean reduction among matching year collections.
def reduce_by_join(img):
  year_col = ee.ImageCollection.fromImages(ee.Image(img).get('year_matches'))
  return year_col.reduce(ee.Reducer.mean()).rename('NBR').set(
      'system:time_start',
      ee.Image(img).date().update(month=8, day=1).millis())

# Apply the `reduce_by_join` function to the list of annual images in the
# properties of the join collection.
landsat_col = join_col.map(reduce_by_join)

In [153]:
reduce_landsat = create_reduce_region_function(
    geometry=point, reducer=ee.Reducer.first(), scale=30, crs='EPSG:3310')

nbr_stat_fc = ee.FeatureCollection(landsat_col.map(reduce_landsat)).filter(
    ee.Filter.notNull(landsat_col.first().bandNames()))

In [154]:
nbr_dict = fc_to_dict(nbr_stat_fc).getInfo()
nbr_df = pd.DataFrame(nbr_dict)
display(nbr_df.head())
print(nbr_df.dtypes)

,NBR,millis,system:index
0,0.215239,1375343817522,1_1_LC08_170060_20130505
1,0.094268,1406879685845,1_1_LC08_170060_20140508
2,0.518093,1438415652094,1_1_LC08_170060_20150511
3,0.294400,1470038080761,1_1_LC08_170060_20160513
4,0.471646,1501574066170,1_1_LC08_170060_20170430


NBR             float64
millis            int64
system:index     object
dtype: object


In [155]:
nbr_df = add_date_info(nbr_df)
nbr_df.head(5)

,NBR,millis,system:index,Timestamp,Year,Month,Day,DOY
0,0.215239,1375343817522,1_1_LC08_170060_20130505,2013-08-01 07:56:57.522,2013,8,1,213
1,0.094268,1406879685845,1_1_LC08_170060_20140508,2014-08-01 07:54:45.845,2014,8,1,213
2,0.518093,1438415652094,1_1_LC08_170060_20150511,2015-08-01 07:54:12.094,2015,8,1,213
3,0.294400,1470038080761,1_1_LC08_170060_20160513,2016-08-01 07:54:40.761,2016,8,1,214
4,0.471646,1501574066170,1_1_LC08_170060_20170430,2017-08-01 07:54:26.170,2017,8,1,213


In [156]:
alt.Chart(nbr_df).mark_line().encode(
    x=alt.X('Timestamp:T', title='Date'),
    y='NBR:Q',
    tooltip=[
             alt.Tooltip('Timestamp:T', title='Date'),
             alt.Tooltip('NBR:Q')
             ]).properties(width=600, height=300).interactive()

alt.Chart(...)